In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = 'OPEN API KEY HERE'

## Load the dataset

In [3]:
df = pd.read_excel('https://raw.githubusercontent.com/ChristineChagas/First_AI_Day4_Act1/main/SupplyChainX_Parcel_Data.xlsx')

In [5]:
df.sample(5)

,Parcel_ID,Weight (kg),Length (cm),Width (cm),Height (cm),Shipping_Type,Parcel_Type,Contents_Description,Declared_Value (PHP),Packaging_Type,...,Duties_Taxes_Paid,Country_of_Origin,Special_Handling_Instructions,Insurance_Status,Weather_Condition,Traffic_Condition,Packaging_Recommendations,Delivery_Priority,Return_Status,Return_Reason
4,5,10,20,15,12,Standard,Envelope,Documents,3000,Polybag,...,50,Philippines,Keep dry,Insured,Cloudy,Light,"Use eco-friendly packaging, light protection",Low,No,NaN
0,1,12,30,20,25,Standard,Box,Furniture,15000,Cardboard,...,50,Philippines,Handle with care,Insured,Rainy,Moderate,"Use waterproof packaging, reinforce box for he...",High,No,NaN
6,7,15,35,25,30,Express,Box,Books,8000,Wooden Box,...,20,Philippines,Handle carefully,Insured,Clear,Moderate,Ensure proper sealing and waterproof packaging,Medium,No,NaN
12,13,7,40,28,35,Standard,Box,Jewelry,25000,Cardboard,...,150,Philippines,"Fragile, keep in cool place",Insured,Sunny,Low,Use protective packaging for fragile items,Medium,No,NaN
13,14,14,32,20,25,Standard,Box,Food,15000,Wooden Box,...,100,Philippines,"Fragile, keep dry",Not Insured,Cloudy,Moderate,Use protective packaging for fragile items,High,No,NaN


## Understanding the Dataset

In [7]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info,"\n")


Parcel_ID                          int64
Weight (kg)                        int64
Length (cm)                        int64
Width (cm)                         int64
Height (cm)                        int64
Shipping_Type                     object
Parcel_Type                       object
Contents_Description              object
Declared_Value (PHP)               int64
Packaging_Type                    object
Sender_Name                       object
Sender_Address                    object
Sender_Contact                    object
Sender_Email                      object
Receiver_Name                     object
Receiver_Address                  object
Receiver_Contact                  object
Receiver_Email                    object
Tracking_Number                   object
Pickup_Date_Time                  object
Estimated_Delivery_Date           object
Actual_Delivery_Date              object
Delivery_Status                   object
Current_Location                  object
Delivery_Method 

In [8]:
# Identify Categorical and Numerical Features

categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)


Categorical Features: Index(['Shipping_Type', 'Parcel_Type', 'Contents_Description',
       'Packaging_Type', 'Sender_Name', 'Sender_Address', 'Sender_Contact',
       'Sender_Email', 'Receiver_Name', 'Receiver_Address', 'Receiver_Contact',
       'Receiver_Email', 'Tracking_Number', 'Pickup_Date_Time',
       'Estimated_Delivery_Date', 'Actual_Delivery_Date', 'Delivery_Status',
       'Current_Location', 'Delivery_Method', 'Proof_of_Delivery',
       'Shipping_Carrier', 'Shipping_Method', 'Tracking_URL',
       'Customs_Declaration', 'Country_of_Origin',
       'Special_Handling_Instructions', 'Insurance_Status',
       'Weather_Condition', 'Traffic_Condition', 'Packaging_Recommendations',
       'Delivery_Priority', 'Return_Status'],
      dtype='object')
Numerical Features: Index(['Parcel_ID', 'Weight (kg)', 'Length (cm)', 'Width (cm)', 'Height (cm)',
       'Declared_Value (PHP)', 'Shipping_Cost (PHP)', 'Duties_Taxes_Paid',
       'Return_Reason'],
      dtype='object')


In [11]:
# Get Unique Values for Categorical Features with Limitation

def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist() # get unique values and turn them into lists
    if len(uniques) > max_display: # if there are more than 'max_display' unique values
        return uniques[:max_display] + ["..."]
    return uniques
unique_values = {col: get_unique_values(col) for col in categorical_features}

print("Unique Values for Categorical Features:")
for col, values in unique_values.items():
    print(f" - {col}: {values}")
print()


Unique Values for Categorical Features:
 - Shipping_Type: ['Standard', 'Express']
 - Parcel_Type: ['Box', 'Envelope']
 - Contents_Description: ['Furniture', 'Documents', 'Electronics', 'Medical Supplies', 'Apparel', 'Books', 'Clothing', 'Toys', 'Fragile Items', 'Health Products', '...']
 - Packaging_Type: ['Cardboard', 'Polybag', 'Wooden Box', 'Plastic', 'Glass']
 - Sender_Name: ['Juan Dela Cruz', 'Ana Dela Cruz', 'Mario Lopez', 'Pedro Martinez', 'Maria Lopez', 'Jose Torres', 'Liza Garcia', 'Carlos Rivera', 'Rita Cruz', 'Paul Martinez', '...']
 - Sender_Address: ['123 Rizal St, Makati', '789 Aurora Blvd, QC', '456 Mango St, Cebu', '789 Bay St, Taguig', '123 Bonifacio St, QC', '456 Green St, Manila', '12 Main St, Pasig', '85 Rizal Ave, Cebu', '23 Quezon Ave, Makati', '15 Rosario St, Pasay', '...']
 - Sender_Contact: ['+63 917 123 4567', '+63 917 234 5678', '+63 917 567 8901', '+63 917 345 6789', '+63 919 234 5678', '+63 918 234 5678', '+63 919 345 6789', '+63 920 567 8901', '+63 920 345

In [12]:
# Get Descriptive Statistics for Numerical Features


numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
        Parcel_ID  Weight (kg)  Length (cm)  Width (cm)  Height (cm)  \
count   20.00000     20.00000    20.000000   20.000000    20.000000   
mean    10.50000     11.00000    29.050000   20.850000    24.750000   
std      5.91608      6.30789    12.028366    9.884997    14.410796   
min      1.00000      3.00000    10.000000    8.000000     8.000000   
25%      5.75000      6.00000    20.000000   14.750000    14.000000   
50%     10.50000     10.00000    30.000000   18.000000    20.000000   
75%     15.25000     14.25000    36.250000   28.000000    31.250000   
max     20.00000     25.00000    50.000000   45.000000    60.000000   

       Declared_Value (PHP)  Shipping_Cost (PHP)  Duties_Taxes_Paid  \
count             20.000000            20.000000          20.000000   
mean            9650.000000           920.000000          70.000000   
std             8006.740581           428.092464          41.927255   
min             1000.000000 

## Generating the Template

In [17]:
def generate_template(df,
                       column_info,
                       categorical_features,
                       numerical_features,
                       unique_values,
                       numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   {stat_name}: {value}\n"


    # Define the system prompt
    system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly."""


    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None



In [18]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)

"The parcel with ID {Parcel_ID}, weighing {Weight (kg)} kg and measuring {Length (cm)} cm in length, {Width (cm)} cm in width, and {Height (cm)} cm in height, is categorized as a {Shipping_Type} {Parcel_Type}. It contains {Contents_Description} and is packaged in a {Packaging_Type}. The declared value of this parcel is {Declared_Value (PHP)} PHP. It was sent by {Sender_Name} from {Sender_Address}, who can be contacted at {Sender_Contact} or via email at {Sender_Email}. The parcel is addressed to {Receiver_Name} at {Receiver_Address}, with the receiver's contact being {Receiver_Contact} and email {Receiver_Email}. The tracking number for this shipment is {Tracking_Number}, which was picked up on {Pickup_Date_Time} and is estimated to be delivered by {Estimated_Delivery_Date}. The actual delivery date was {Actual_Delivery_Date}, and the delivery status is currently {Delivery_Status}. The parcel is located at {Current_Location} and is being delivered via {Delivery_Method} by {Shipping_Car

## Populating the Template with Actual Values

In [21]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
  row_dict = row.to_dict()
  for key, value in row_dict.items():
    if pd.isna(value):
      row_dict[key] = 'N/A'


    # Generate the populated template per row
  paragraph = template.format(**row_dict)

  return paragraph

In [22]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [23]:
df['paragraph'][0]

'"The parcel with ID 1, weighing 12 kg and measuring 30 cm in length, 20 cm in width, and 25 cm in height, is categorized as a Standard Box. It contains Furniture and is packaged in a Cardboard. The declared value of this parcel is 15000 PHP. It was sent by Juan Dela Cruz from 123 Rizal St, Makati, who can be contacted at +63 917 123 4567 or via email at juandelacruz@email.com. The parcel is addressed to Maria Santos at 456 Quezon Ave, Manila, with the receiver\'s contact being +63 918 234 5678 and email mariasantos@email.com. The tracking number for this shipment is TRACK123456, which was picked up on 2024-11-01 09:00 and is estimated to be delivered by 2024-11-05. The actual delivery date was 2024-11-04, and the delivery status is currently Delivered. The parcel is located at Makati and is being delivered via Air by J&T Express. The shipping cost incurred is 850 PHP, and the shipping method used is Ground. For tracking, visit https://tracking.url/123456. The customs declaration numbe

## Saving the Populated Templates (Natural Language Texts)

In [24]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")



In [25]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [26]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')
save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='nlg.pdf')


All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

